In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf

In [ ]:
# Hyperparameters
batch_size = 180
num_classes = 10
epochs = 10

In [ ]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train[:10000,:,:,:]
x_test = x_test[:1000,:,:,:]

y_train = y_train[:10000]
y_test = y_test[:1000]

# convert to one hot encoing 
Y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
Y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

X_train = tf.keras.layers.UpSampling2D(size=(2, 2))(x_train)
X_train = tf.keras.layers.UpSampling2D(size=(2, 2))(X_train)


X_test = tf.keras.layers.UpSampling2D(size=(2, 2))(x_test)
X_test = tf.keras.layers.UpSampling2D(size=(2, 2))(X_test)


print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

170508288/170498071 [==============================] - 2s 0us/step
(10000, 128, 128, 3)
(1000, 128, 128, 3)
(10000, 10)
(1000, 10)


In [ ]:
inception_v3 = Sequential()

pretrained_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3),
    pooling='max',
    classes=10,
    classifier_activation="softmax",
)

for layer in pretrained_model.layers:
    layer.trainable=False

inception_v3.add(pretrained_model)
inception_v3.add(Flatten())
inception_v3.add(Dense(128, activation='relu'))
inception_v3.add(Dense(10, activation='softmax'))

87924736/87910968 [==============================] - 0s 0us/step


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(X_train)

In [ ]:
inception_v3.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# I have not used it as it was having no effect on traning 
from keras.callbacks import ModelCheckpoint, CSVLogger
ckpt = ModelCheckpoint('saved_model/inception_v3.hdf5')
csv = CSVLogger('saved_model/inception_v3_log.csv')

In [ ]:
history = inception_v3.fit(datagen.flow(X_train, Y_train, batch_size), validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size)
inception_v3.save_weights('saved_model/10epochs_inception_v3_cifar10.h5')

Epoch 1/10
56/56 [==============================] - 293s 5s/step - loss: 44.5730 - accuracy: 0.1495 - val_loss: 6.7287 - val_accuracy: 0.1610
Epoch 2/10
56/56 [==============================] - 289s 5s/step - loss: 2.6548 - accuracy: 0.1106 - val_loss: 2.2964 - val_accuracy: 0.1010
Epoch 3/10
56/56 [==============================] - 289s 5s/step - loss: 2.3070 - accuracy: 0.1044 - val_loss: 2.2996 - val_accuracy: 0.1020
Epoch 4/10
56/56 [==============================] - 290s 5s/step - loss: 2.3039 - accuracy: 0.1032 - val_loss: 2.3039 - val_accuracy: 0.1020
Epoch 5/10
56/56 [==============================] - 288s 5s/step - loss: 2.3025 - accuracy: 0.1032 - val_loss: 2.2975 - val_accuracy: 0.1040
Epoch 6/10
56/56 [==============================] - 291s 5s/step - loss: 2.3030 - accuracy: 0.1051 - val_loss: 2.2953 - val_accuracy: 0.1070
Epoch 7/10
56/56 [==============================] - 288s 5s/step - loss: 2.3018 - accuracy: 0.1000 - val_loss: 2.2990 - val_accuracy: 0.1020
Epoch 8/10
5

OSError: ignored

In [ ]:
train_mse = inception_v3.evaluate(X_train, Y_train, verbose=0)
test_mse = inception_v3.evaluate(X_test, Y_test, verbose=0)

In [ ]:
print('Train loss : %.3f %% , Test loss : %.3f %%' % (train_mse[0]*100, test_mse[0]*100))
print('Train accuracy : %.3f %% , Test accuracy : %.3f %%' % (train_mse[1]*100, test_mse[1]*100))

NameError: ignored